In [1]:
import os
from dataset_processing import remove_quotes_from_file, filter_by_max_words, clean_text, write_training_file, convert_stopwords_to_lower_case
import pandas as pd
import platform
import json

In [2]:
# set the filepaths for the data
operating_system = platform.system()
if operating_system == 'Windows':
    raise Exception("not used on Windows yet")
elif operating_system == 'Linux':
    train_dir = "/run/user/1000/gvfs/google-drive:host=gmail.com,user=cjameswalmsley/0AG4X1_cOZQRnUk9PVA/0B24X1_cOZQRnVjZEemxXd3BNSTQ/1SSfZglixkSB5aj0qMv2BsDojVhxNOv0o/11BZhonitY-Jfw5xqM1xAl5TjJPRDRKHr/1OyCmWPf_7PH1bCc-5GLvBdo-gvEvJsiP/1dgL8C9ydesEz3jM28e55yGR9PECRtjaA/1_z0x1Ym8ceREizMlb1mk29msDhvfy2xg/1951JmEhJWAQjilgGc7T00-z9bbiJwNM3"
elif operating_system == 'Darwin': #macOS
    train_dir = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/training"
else:
    raise Exception("unsupported OS")

pre_training_dir = "pre_training"
statements_filepath = os.path.join(pre_training_dir, "nyc_statements.txt")
train_filename = "declarative_sentences_train_gemma3:1b_20250707_174634.tsv"
train_filepath = os.path.join(train_dir, train_filename)
statements_json_filepath = os.path.join(pre_training_dir, "nyc_statements_response.json")
questions_json_filepath = os.path.join(pre_training_dir, "nyc_questions_response.json")
statements_filepath = os.path.join(pre_training_dir, "nyc_statements.txt")
questions_filepath = os.path.join(pre_training_dir, "nyc_questions.txt")

In [3]:
#load and clean the nyc training data
train_filepath = remove_quotes_from_file(train_filepath)
train_df = pd.read_csv(train_filepath, sep="\t")
train_df = train_df.dropna()
filtered_train_df = filter_by_max_words(train_df, max_words=20)
filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
filtered_train_df["statement"] = clean_text(filtered_train_df["statement"], False)
filtered_train_df.reset_index(drop=True, inplace=True)
filtered_train_df.info()

Cleaned data saved to /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/training/declarative_sentences_train_gemma3:1b_20250707_174634_cleaned.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 798 non-null    object
 1   title              798 non-null    object
 2   question           798 non-null    object
 3   answer             798 non-null    object
 4   response_question  798 non-null    object
 5   response_answer    798 non-null    object
 6   statement          798 non-null    object
dtypes: object(7)
memory usage: 43.8+ KB


/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_24017/1635968978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_24017/1635968978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_2401

In [4]:
write_training_file(filtered_train_df, statements_filepath)
with open(statements_filepath, "r") as training_file:
    training_lines = training_file.readlines()

file created: pre_training/nyc_statements.txt


In [5]:
questions = filtered_train_df["question"]
with open(questions_filepath, "w") as questions_file:
    for question in questions:
        questions_file.write(f"{question}\n")
with open(questions_filepath, "r") as questions_file:
    questions_lines = questions_file.readlines()
for line in questions_lines[:5]:
    print(line)

What city in the United States has the highest population?

In what city is the United Nations based?

What city has been called the cultural capital of the world?

What American city welcomes the largest number of legal immigrants?

The major gateway for immigration has been which US city?



##Find a selection of training samples from the statements and questions
* Take the contents of the 2 files and ask an LLM to categorise the questions and statements
* The following prompt can be used to categorise:
> TASK: categorise the provided list of sentences by grammatical structure
RESPONSE FORMAT: provide the response in JSON format with each row formatted as {"category":"<INSERT CATEGORY>", "sentence":"<INSERT SENTENCE>"}
>

In [6]:
with open(statements_json_filepath, 'r') as statements_json_file:
    statements_json_lines = statements_json_file.readlines()
statements = [json.loads(line) for line in statements_json_lines if line.strip()]
print(statements[:5])
statements_df = pd.DataFrame(statements)
statements_df = statements_df[statements_df["category"] != "INTERROGATIVE"]
statements_df.groupby('category').count()

[{'category': 'SUBJECT_LINKING_VERB_COMPLEMENT', 'sentence': 'York is a city in the United States with the highest population'}, {'category': 'PASSIVE_CONSTRUCTION', 'sentence': 'The United Nations is headquartered in New York'}, {'category': 'PASSIVE_CONSTRUCTION', 'sentence': 'New York has been called the cultural capital of the world'}, {'category': 'SUBJECT_VERB_OBJECT', 'sentence': 'New York welcomes the largest number of legal immigrants'}, {'category': 'SUBJECT_LINKING_VERB_COMPLEMENT', 'sentence': 'The major gateway for immigration has been New York City'}]


,sentence
category,
ATTRIBUTE_IS_VALUE,117
EXISTENTIAL_SENTENCE,26
PASSIVE_CONSTRUCTION,185
PREPOSITIONAL_PHRASE_FRONTING,33
SUBJECT_LINKING_VERB_COMPLEMENT,148
SUBJECT_VERB_OBJECT,270


In [7]:
with open(questions_json_filepath, 'r') as questions_json_file:
    questions_json_lines = questions_json_file.readlines()
questions = [json.loads(line) for line in questions_json_lines if line.strip()]
print(questions[:5])
questions_df = pd.DataFrame(questions)
questions_df.groupby('category').count()

[{'category': 'WHAT_QUESTION', 'sentence': 'What city in the United States has the highest population?'}, {'category': 'WHERE_IN_ON_AT_QUESTION', 'sentence': 'In what city is the United Nations based?'}, {'category': 'WHAT_QUESTION', 'sentence': 'What city has been called the cultural capital of the world?'}, {'category': 'WHAT_QUESTION', 'sentence': 'What American city welcomes the largest number of legal immigrants?'}, {'category': 'WHICH_QUESTION', 'sentence': 'The major gateway for immigration has been which US city?'}]


,sentence
category,
HOW_QUESTION,179
WHAT_QUESTION,400
WHEN_QUESTION,18
WHERE_IN_ON_AT_QUESTION,126
WHICH_QUESTION,69
WHO_QUESTION,25


In [8]:
#add the categories to the filtered_train_df
def add_categories_to_df(df, category_column, sentences):
    categories = []
    new_column_name = category_column + "_category"
    for sentence in df[category_column]:
        matching_row = sentences[sentences["sentence"] == sentence]
        if not matching_row.empty:
            categories.append(matching_row["category"].values[0])
        else:
            categories.append(None)
    df[new_column_name] = categories
    return df

In [9]:
filtered_train_df_with_categories = add_categories_to_df(filtered_train_df, "statement", statements_df)
filtered_train_df_with_categories = add_categories_to_df(filtered_train_df_with_categories, "question", questions_df)
filtered_train_df_with_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  798 non-null    object
 1   title               798 non-null    object
 2   question            798 non-null    object
 3   answer              798 non-null    object
 4   response_question   798 non-null    object
 5   response_answer     798 non-null    object
 6   statement           798 non-null    object
 7   statement_category  608 non-null    object
 8   question_category   793 non-null    object
dtypes: object(9)
memory usage: 56.2+ KB


/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_24017/530804637.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = categories
/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_24017/530804637.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name] = categories


In [10]:
filtered_train_df_with_categories.dropna(inplace=True)
filtered_train_df_with_categories.reset_index(drop=True, inplace=True)
filtered_train_df_with_categories

/var/folders/cv/z7xz0c3n3nd4c26hj7g01n7w0000gn/T/ipykernel_24017/2512786238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df_with_categories.dropna(inplace=True)


,id,title,question,answer,response_question,response_answer,statement,statement_category,question_category
0,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? In what city is the United Nations based,New York,The United Nations is headquartered in New York,PASSIVE_CONSTRUCTION,WHERE_IN_ON_AT_QUESTION
1,56ce304daab44d1400b88510,New_York_City,What city has been called the cultural capital...,New York,? What city has been called the cultural capit...,New York,New York has been called the cultural capital ...,PASSIVE_CONSTRUCTION,WHAT_QUESTION
2,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,New York,New York welcomes the largest number of legal ...,SUBJECT_VERB_OBJECT,WHAT_QUESTION
3,56cf5d41aab44d1400b89130,New_York_City,The major gateway for immigration has been whi...,New York City,? The major gateway for immigration has been w...,New York City,The major gateway for immigration has been New...,SUBJECT_LINKING_VERB_COMPLEMENT,WHICH_QUESTION
4,56cf5d41aab44d1400b89131,New_York_City,The most populated city in the United States i...,New York City,? The most populated city in the United States...,New York City,The most populated city in the United States i...,SUBJECT_LINKING_VERB_COMPLEMENT,WHICH_QUESTION
...,...,...,...,...,...,...,...,...,...
599,56d1204617492d1400aab9fd,New_York_City,In what borough is the New York City Hall found?,Manhattan,? In what borough is the New York City Hall found,Manhattan,The New York City Hall is located in Manhattan,PASSIVE_CONSTRUCTION,WHERE_IN_ON_AT_QUESTION
600,56d1218c17492d1400aaba1e,New_York_City,What ZIP code was responsible for the greatest...,10021,? What ZIP code was responsible for the greate...,10021,The ZIP code 10021 was responsible for the gre...,SUBJECT_LINKING_VERB_COMPLEMENT,WHAT_QUESTION
601,56d1218c17492d1400aaba1f,New_York_City,How much money in cents does New York City rec...,83,? How much money in cents does New York City r...,83,New York City receives 83 cents for every doll...,SUBJECT_VERB_OBJECT,HOW_QUESTION
602,56d1218c17492d1400aaba20,New_York_City,How much more money does the city give to the ...,$11 billion,? How much more money does the city give to th...,11 billion,The city gives 11 billion more to the state of...,SUBJECT_VERB_OBJECT,HOW_QUESTION


In [11]:
print(filtered_train_df_with_categories.groupby(["question_category"]).count())
print(filtered_train_df_with_categories.groupby(["statement_category"]).count())

                          id  title  question  answer  response_question  \
question_category                                                          
HOW_QUESTION             133    133       133     133                133   
WHAT_QUESTION            329    329       329     329                329   
WHEN_QUESTION              1      1         1       1                  1   
WHERE_IN_ON_AT_QUESTION   69     69        69      69                 69   
WHICH_QUESTION            52     52        52      52                 52   
WHO_QUESTION              20     20        20      20                 20   

                         response_answer  statement  statement_category  
question_category                                                        
HOW_QUESTION                         133        133                 133  
WHAT_QUESTION                        329        329                 329  
WHEN_QUESTION                          1          1                   1  
WHERE_IN_ON_AT_QUESTI

In [12]:
# Function to randomly select a sample of statements and questions from each category
def select_random_samples(df, category_column, sample_size=3):
    samples = []
    for category in df[category_column].unique():
        category_samples = df[df[category_column] == category].sample(n=min(sample_size, len(df[df[category_column] == category])), random_state=42)
        samples.append(category_samples)
    return pd.concat(samples).reset_index(drop=True)

sample_statements_df = select_random_samples(filtered_train_df_with_categories, "statement_category")
sample_questions_df = select_random_samples(filtered_train_df_with_categories, "question_category")

In [13]:
print(sample_questions_df.groupby("question_category").count())
print(sample_statements_df.groupby("statement_category").count())

                         id  title  question  answer  response_question  \
question_category                                                         
HOW_QUESTION              3      3         3       3                  3   
WHAT_QUESTION             3      3         3       3                  3   
WHEN_QUESTION             1      1         1       1                  1   
WHERE_IN_ON_AT_QUESTION   3      3         3       3                  3   
WHICH_QUESTION            3      3         3       3                  3   
WHO_QUESTION              3      3         3       3                  3   

                         response_answer  statement  statement_category  
question_category                                                        
HOW_QUESTION                           3          3                   3  
WHAT_QUESTION                          3          3                   3  
WHEN_QUESTION                          1          1                   1  
WHERE_IN_ON_AT_QUESTION      

In [14]:
statements_mask = filtered_train_df_with_categories["statement"].isin(sample_statements_df["statement"])
statements_mask.value_counts()

statement
False    586
True      18
Name: count, dtype: int64

In [15]:
questions_mask = filtered_train_df_with_categories["question"].isin(sample_questions_df["question"])
questions_mask.value_counts()

question
False    588
True      16
Name: count, dtype: int64

In [20]:
filtered_samples_df = filtered_train_df_with_categories[statements_mask | questions_mask].copy()
filtered_samples_df.reset_index(drop=True, inplace=True)
filtered_samples_df["statement"] = filtered_samples_df["statement"].apply(lambda x: convert_stopwords_to_lower_case(x))
filtered_samples_df

,id,title,question,answer,response_question,response_answer,statement,statement_category,question_category
0,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? In what city is the United Nations based,New York,the United Nations is headquartered in New York,PASSIVE_CONSTRUCTION,WHERE_IN_ON_AT_QUESTION
1,56cf9f16234ae51400d9be30,New_York_City,How many school and universities are in NYC?,120,? How many school and universities are in NYC,120,there are 120 school and universities in NYC,EXISTENTIAL_SENTENCE,HOW_QUESTION
2,56ce33aaaab44d1400b8856a,New_York_City,What was the name of the Lenape homeland?,Lenapehoking,? What was the name of the Lenape homeland,Lenapehoking,the Lenapehoking was the Lenape homeland,SUBJECT_LINKING_VERB_COMPLEMENT,WHAT_QUESTION
3,56ce34c7aab44d1400b88595,New_York_City,Who commanded the Spanish expedition?,Estêvão Gomes,? Who commanded the Spanish expedition,Estevao Gomes,Estevao Gomes commanded the Spanish expedition,SUBJECT_VERB_OBJECT,WHO_QUESTION
4,56cfb206234ae51400d9be91,New_York_City,What was the myth that Manhattan was bought fo...,glass beads,? What was the myth that Manhattan was bought ...,glass beads,General Peter Minuit acquired Manhattan Island...,SUBJECT_VERB_OBJECT,WHAT_QUESTION
5,56cfb5cb234ae51400d9beb7,New_York_City,When was as African burial ground discovered a...,1990s,? When was as African burial ground discovered...,1990s,after the building of new courthouse African b...,PREPOSITIONAL_PHRASE_FRONTING,WHEN_QUESTION
6,56cee30faab44d1400b88bf3,New_York_City,What notable catastrophe took place under the ...,Great Fire of New York,? What notable catastrophe took place under th...,Great Fire of New York,Great Fire of New York took place under the Br...,SUBJECT_VERB_OBJECT,WHAT_QUESTION
7,56cee638aab44d1400b88c34,New_York_City,About how many people died during the Draft Ri...,120,? About how many people died during the Draft ...,120,during the Draft Riots of 1863 approximately 1...,PREPOSITIONAL_PHRASE_FRONTING,HOW_QUESTION
8,56cee7edaab44d1400b88c67,New_York_City,"In the early 1920s, what was the second most h...",London,? In the early 1920s what was the second most ...,London,in the early 1920s London was the second most ...,PREPOSITIONAL_PHRASE_FRONTING,WHAT_QUESTION
9,56cfdde6234ae51400d9bfa8,New_York_City,How many leader terrorists of Al Quada were in...,10,? How many leader terrorists of Al Quada were ...,10,the number of leader terrorists of Al Quada di...,ATTRIBUTE_IS_VALUE,HOW_QUESTION


In [21]:
pre_training_filename = "pre_training_nyc_samples"
pre_training_filepath = os.path.join(pre_training_dir, pre_training_filename)
if not os.path.exists(pre_training_dir):
    os.makedirs(pre_training_dir)
pre_training_df = filtered_samples_df[["statement_category", "statement", "question_category", "response_question", "response_answer"]].copy()
pre_training_df.to_csv(pre_training_filepath, sep="\t", index=False)
with open(pre_training_filepath, "r") as pre_training_file:
	pre_training_lines = pre_training_file.readlines()
pre_training_lines[:5]

['statement_category\tstatement\tquestion_category\tresponse_question\tresponse_answer\n',
 'PASSIVE_CONSTRUCTION\tthe United Nations is headquartered in New York\tWHERE_IN_ON_AT_QUESTION\t? In what city is the United Nations based\tNew York\n',
 'EXISTENTIAL_SENTENCE\tthere are 120 school and universities in NYC\tHOW_QUESTION\t? How many school and universities are in NYC\t120\n',
 'SUBJECT_LINKING_VERB_COMPLEMENT\tthe Lenapehoking was the Lenape homeland\tWHAT_QUESTION\t? What was the name of the Lenape homeland\tLenapehoking\n',
 'SUBJECT_VERB_OBJECT\tEstevao Gomes commanded the Spanish expedition\tWHO_QUESTION\t? Who commanded the Spanish expedition\tEstevao Gomes\n']

In [27]:
pre_training_automatic_filename = "pre_training_nyc_automatic"
pre_training_automatic_filepath = os.path.join(pre_training_dir, pre_training_automatic_filename)
with open(pre_training_automatic_filepath, "w") as pre_training_automatic_file:
    pre_training_automatic_file.write("#statements\n")
    #write the statements sorted by statement category
    for category, group in pre_training_df.groupby("statement_category"):
        pre_training_automatic_file.write(f"#\n#category: {category}\n")
        for statement in group["statement"]:
            pre_training_automatic_file.write(f"{statement}\n")
    pre_training_automatic_file.write("#\n#questions\n")
    #write the questions sorted by question category
    for category, group in pre_training_df.groupby("question_category"):
        pre_training_automatic_file.write(f"#\n#category: {category}\n")
        for question in group["response_question"]:
            pre_training_automatic_file.write(f"{question}\n")